First, let's import the packages that will be used throughout this analysis.

In [ ]:
import pandas as pd

from processing import cleaners, io, util

RACE_YEAR = 2019

Then change a setting so we can see the results of variable assignments.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last_expr_or_assign'

This notebook assumes we have already run a scrapy spider to collect race data like so:

```python
from processing.scrapers import scrape_for_pandas
scrape_for_pandas(RACE_YEAR)
```

## Load, process, and clean split info scraped by the spider.

### Load the data (as scraped by the spider) into a pandas DataFrame.

In [ ]:
df_split_info_raw = io.load_df_split_info_raw(RACE_YEAR)

In [ ]:
df_split_info = cleaners.process_df_split_info(df_split_info_raw)

# Debug
# df_split_info
# df_splits.dtypes

### Apply some Leadville-specific processing to the split info

In [ ]:
# (present in 2021, gone in 2022, and never very useful)
df_split_info.drop(index='Kick to Finish 0.8 Miles Left', errors='ignore', inplace=True)

# Extract each aid station name from its 'label' string.
# NOTE: This isn't really necessary, just aesthetic. Move to plotting script?
df_split_info['name'] = df_split_info.index.to_series(
  ).str.extract(r'Start to (.+)', expand=False,
  # If the split name doesn't match the regex, just use as-is.
  ).fillna(df_split_info.index.to_series())

# Manually add cutoff info (found elsewhere on the web)
df_split_info['cutoff_hr'] = [3.75, 6., 7.5, 9.5, 12., 14., None, 18., 21.25, 23., 26.5, 30.]

df_split_info

# Debug
# df_splits.dtypes
# type(df_splits['cutoff_hr'].iloc[6])  # np.float64

### (Optional) write cleaned split info to file

```python
import os

# TODO: implement this
# io.save_df_split_info(df_split_info, RACE_YEAR)

# dir_out = settings.CLEAN_RACE_DATA_DIR
dir_out = io.get_clean_race_data_dir(RACE_YEAR)
if not os.path.exists(dir_out):
  os.makedirs(dir_out)

# df_split_info.to_csv(os.path.join(dir_out, settings.SPLIT_INFO_FNAME),
df_split_info.to_csv(os.path.join(dir_out, io.SPLIT_INFO_FNAME),
  columns=['name', 'distance_mi', 'cutoff_hr'],
  index=True)
```

## Load, process, and clean the athlete split data scraped by the spider

The Athlinks spider saves split data for each athlete. Load all such split data from the spider-produced `json` file into a `pandas.DataFrame`.

In [ ]:
df_split_times_raw = io.load_df_split_times_raw(RACE_YEAR)

util.df_td_fmt(df_split_times_raw)

Notice that the columns (athletes) are presented in the order the Athlinks website spider returned them. This isn't particularly useful for spotting outliers or unreasonable performances. It would be easier to spot bad data if athletes with similar performances were grouped together. To fix this, we can group the athletes based on the furthest split they each recorded, then sort athletes within each group by their split times.

In [ ]:
# Load the split info that's been processed and cleaned
# NOTE: no need to reload, I created it above.
# df_split_info = io.load_df_split_info_clean(settings.CLEAN_RACE_DATA_DIR)
# print(df_split_info_clean)

# df_split_times = util.load_athlete_split_times(clean=False)
# df_split_times = cleaners.sort_df_split_data(
#   io.load_df_split_times_raw(settings.RAW_RACE_DATA_DIR),
#   df_split_info_clean)

df_split_times = cleaners.sort_df_split_data(df_split_times_raw, df_split_info)

util.display_full_df(util.df_td_fmt(df_split_times))

In [ ]:

# Optional: Print to a file so
# I can inspect the whole thing for abnormalities (alt+z to disable wrap).
# Mostly looking for anomalous finish line splits when a runner DNFs and 
# returns to Leadville.
# NOTE: Cannot view this in vscode if it goes over 10k characters wide.
# Sublime text can do it.
# util.full_df_to_file(
#   util.df_td_fmt(df_split_times),
#   os.path.join(settings.RAW_RACE_DATA_DIR, 'df_all.txt')
# )

### Remove wonky runner data, as determined by manual inspection.

Viewing the data this way highlights erroneous split data. For example, look at the first column, containing the splits for Charles Corfield. The uncleaned split data suggests that Charles won the entire race, because his finish line split is the fastest. But looking more closely, it seems he dropped out of the race after the split labeled `56.5mi_Start to Hope Pass In`, and his bib triggered the sensor at the finish line when he came to watch the 2019 champion, Ryan Smith, bring it home. Something similar seemed to happen with Vineer Bhansali.

In [ ]:
util.df_td_fmt(
  df_split_times[['Charles Corfield', 'Ryan Smith', 'Vineer Bhansali']]
)

To cleanse the data of these erroneous splits, we just need to set them to a null value.

In [ ]:
df_split_times.loc['Full Course', 'Charles Corfield'] = pd.NA
df_split_times.loc['Full Course', 'Vineer Bhansali'] = pd.NA

util.df_td_fmt(
  df_split_times[['Charles Corfield', 'Ryan Smith', 'Vineer Bhansali']]
)

Another type of error can happen when a split is triggered erroneously early, for whatever reason. This type of erroneous data doesn't necessarily jump out upon visual inspection. One way to check for it is to see if any athletes data includes a negative point-to-point time.

In [ ]:
# df_split_times.diff().iloc[1:].apply(lambda col: (col.dt.total_seconds() < 0).any())
# df_split_times.diff().iloc[1:].apply(lambda col: col[~col.isnull()].dt.total_seconds() < 0).all()
# df_split_times.diff().iloc[1:].apply(lambda col: (col.dt.total_seconds() > 0))
# df_split_times.diff().iloc[1:].apply(lambda col: (col.dt.total_seconds() > 0).all())
# df_split_times.apply(lambda col: col.is_monotonic_increasing)

# series_athlete_has_negative_timedelta = df_split_times.diff().apply(
#   lambda col: (col.dt.total_seconds() < 0).any())

series_athlete_has_negative_timedelta = df_split_times.apply(
  lambda col: (col[col.notnull()].dt.total_seconds().diff() < 0).any())

# series_athlete_has_negative_timedelta = df_split_times.apply(
#   lambda col: col.notnull()
# )

util.df_td_fmt(
  df_split_times.loc[:, series_athlete_has_negative_timedelta]
)

It's not immediately clear what is going on with the split data for either of these athletes.

In Willie's case, the erroneously negative elapsed time comes between the splits at 76.9 miles and 87.8 miles. But even if we discarded the split data at 87.8 miles as an obvious misfire, no one could cover the distance from 76.9 miles to the finish in such a short time. But is the error at 76.9 miles or the finish line? Both?

It's impossible to tell which split data is valid or invalid without more context. Let's look at athletes with similar finish times to each athlete with bad data.

In [ ]:
ix = df_split_times.columns.get_loc('Willie Stewart')
util.df_td_fmt(
  df_split_times.iloc[:, ix-5:ix+5]
)

Wow now I have no idea what data is valid.

* It seems likely that Willie Stewart DNFed at 76.9. I just doubt anyone could cover 71.1 -> 100 in 3ish hours,
  so it seems very unlikely that the split at 76.9 is a misfire.
* Chavet Breslin probably DNFed at 76.9 and recorded an erroneous finish.
* Harry Harcrow and Jared Conlin seem to have DNFed at 71.1 but recorded a finish line split near each other.
* Tim Finocchio seemed sketchy at first with how much ground he made up in the later splits, but I
  cannot say anything without busting out the statistics.

Let's cleanse the more likely errors.

In [ ]:
df_split_times.loc['87.8mi_Start to May Queen In': 'Full Course', 'Willie Stewart'] = pd.NA
df_split_times.loc['Full Course', 'Chavet Breslin'] = pd.NA
df_split_times.loc['Full Course', 'Harry Harcrow'] = pd.NA
df_split_times.loc['Full Course', 'Jared Conlin'] = pd.NA

Let's look at the context around Max's finish split.

In [ ]:
ix = df_split_times.columns.get_loc('Max Fulton')
util.df_td_fmt(
  df_split_times.iloc[:, ix-5:ix+5]
)

Immediately, it looks like the split at 43.5 misfired. I think his chip didn't fire on the way out,
but it did fire on the way in. Let's swap those values.

In [ ]:
df_split_times.loc['56.5mi_Start to Hope Pass In', 'Max Fulton'] = df_split_times.loc['43.5mi_Start to Hope Pass Out', 'Max Fulton']
df_split_times.loc['43.5mi_Start to Hope Pass Out', 'Max Fulton'] = pd.NA

In [ ]:

# TODO: Use statistical methods or reasonable assumptions to 
# programmatically flag sketchy split times. Single splits that are
# anomalously fast are invisible to me when I'm visually checking the
# output.

# TODO: Move the functionality below somewhere else - it represents insights
#       that can only be gained after looking at the results of the
#       script so far.

# TODO: Make this programmatic if it needs to be done more than once.
if RACE_YEAR == 2022:
  # Erroneous finish line split after a dnf
  df_split_times.loc['Full Course', 'Mickey Davis'] = pd.NA

  df_split_times.loc['76.9mi_Start to Outward Bound In', 'Timothy Weng'] = pd.NA

  # He would have had to run an unreasonably fast time to get to the next split. 
  df_split_times.loc['29.3mi_Start to Half Pipe Out', 'Adrian Macdonald'] = pd.NA

  # chip reported a negative time from last split
  df_split_times.loc['43.5mi_Start to Hope Pass Out', 'Kris Rugloski'] = pd.NA

  # 05:10:20 split would put him in first. Hmm.
  # Next two splits are pretty fast for his eventual finish time too.
  # Could have been a wonky chip or an aggressive strategy, idk.
  # df_split_times.loc['37.9mi_Start to Twin Lakes Out', 'Michael Nanaszko'] = pd.NA

elif RACE_YEAR == 2021:
  # chip reported a negative time from last split
  df_split_times.loc['37.9mi_Start to Twin Lakes Out', 'Matthew DuBois'] = pd.NA
  # TODO: finish checking.

df_split_times = cleaners.sort_df_split_data(df_split_times, df_split_info)

util.display_full_df(util.df_td_fmt(df_split_times))

# series_athlete_has_negative_timedelta = df_split_times.apply(
#   lambda col: (col[col.notnull()].dt.total_seconds().diff() < 0).any())
# df_split_times.loc[:, series_athlete_has_negative_timedelta]

### (Optional) save inferences for future use

```python
# TODO: Implement?
# util.save_athlete_split_times(df_all_athlete_split_times, clean=True)

dir_out = settings.CLEAN_RACE_DATA_DIR
if not os.path.exists(settings.CLEAN_RACE_DATA_DIR):
  os.makedirs(settings.CLEAN_RACE_DATA_DIR)
io.df_td_to_csv(df_split_times, 
  os.path.join(settings.CLEAN_RACE_DATA_DIR, settings.SPLIT_SECS_FNAME))
```